In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [10]:
import pandas as pd

url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
df = pd.read_csv(url)
df.head()


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [12]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [80]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
df[categorical_features] = df[categorical_features].fillna('NA')
df[numerical_features] = df[numerical_features].fillna(0)


In [81]:
strings = list(df.dtypes[df.dtypes == 'object'].index)
strings

['lead_source', 'industry', 'employment_status', 'location']

In [82]:
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ','_')

In [83]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [84]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,na,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,na,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [85]:
#Question 1
df['industry'].mode()[0]

'retail'

In [86]:
df['industry'].value_counts().head(1)

industry
retail    203
Name: count, dtype: int64

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
df_full_train , df_test = train_test_split(df,test_size = 0.2 ,random_state =42)
df_train ,df_val = train_test_split(df_full_train , test_size = 0.25 ,random_state =42)

In [89]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [90]:
len(df_train), len(df_val) ,len(df_test)
df_full_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1
799,social_media,education,2,71399.0,na,europe,1,0.51,1
380,referral,education,2,47912.0,employed,australia,1,0.04,0
303,paid_ads,healthcare,1,34806.0,employed,europe,4,0.32,1


In [91]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

In [92]:
df_train = df_train.drop('converted',axis =1 )
df_val = df_val.drop('converted',axis =1 )
df_test = df_test.drop('converted',axis =1 )


In [93]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [94]:
df['interaction_count'].corr(df['lead_score'])*100



np.float64(0.988818249691311)

In [95]:
df['number_of_courses_viewed'].corr(df['lead_score'])*100


np.float64(-0.4878998354681267)

In [96]:
df['number_of_courses_viewed'].corr(df['interaction_count'])*100


np.float64(-2.3565222882887937)

In [97]:
df['annual_income'].corr(df['interaction_count'])*100
#Question 2 this is the highest correlation between two varaibles 

np.float64(2.7036472404814353)

In [98]:
#Question 3
from sklearn.metrics import mutual_info_score



In [103]:
numerical = list(df_train.dtypes[df_train.dtypes != 'object'].index)
numerical
y_train = y_train.astype(int)

In [109]:
def mutual_info_converted_score(series):
    return mutual_info_score(series,y_train)

In [113]:
categories = df_train.select_dtypes(include=['object']).columns

mi = df_train[categories].apply(mutual_info_converted_score)

mi = mi.sort_values(ascending = False)
print(round(mi*100,2))
#Question - 3

lead_source          3.54
employment_status    1.29
industry             1.16
location             0.45
dtype: float64


#Logistic Regression

In [117]:
from sklearn.feature_extraction import DictVectorizer

In [118]:
dv = DictVectorizer(sparse = False)

In [122]:
categories = df_train.select_dtypes(include = ['object']).columns.tolist()
numerical =  df_train.select_dtypes(exclude = ['object']).columns.tolist()

train_dict = df_train[categories + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

In [123]:
val_dict = df_val[categories + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dict)

In [114]:
def logistic_regression(xi):
    score = w0

    for j in range(len(w)):
        score = score + xi[j] * w[j]
    result = sigmoid(score)
    return result

In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [131]:
model = LogisticRegression(solver = 'liblinear' , C = 1.0 ,max_iter = 1000,random_state = 42)

In [132]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [133]:
y_val_pred = model.predict(X_val)

In [135]:
accuracy = accuracy_score(y_val,y_val_pred)
print("val accuarcy: " , round(accuracy,2))

val accuarcy:  0.7


In [136]:
#Question 4 = 0.74 answer

In [137]:
#question 5

In [138]:
features_to_test = ['industry','employment_status','lead_socre']

In [139]:
diffs = {}

In [143]:
for feature in features_to_test:
    reduced_features = [f for f in categories + numerical if f != feature]
    dv_reduced = DictVectorizer(sparse =False)
    X_train_reduced = dv_reduced.fit_transform(df_train[reduced_features].to_dict(orient = 'records'))
    X_val_reduced = dv_reduced.transform(df_val[reduced_features].to_dict(orient='records'))
    model_reduced = LogisticRegression(solver = 'liblinear', C = 1.0 ,max_iter =1000,random_state =42)
    model_reduced.fit(X_train_reduced,y_train)
    y_val_pred_reduced = model_reduced.predict(X_val_reduced)
    reduced_accuracy = accuracy_score(y_val,y_val_pred_reduced)
    diff = accuracy-reduced_accuracy
    diffs[feature] =diff
    print(f"Feature: {feature}, Accuracy without: {reduced_accuracy}, Difference: {diff}")
    least_useful = min(diffs, key=diffs.get)
    print("Least useful feature:", least_useful)




Feature: industry, Accuracy without: 0.6996587030716723, Difference: 0.0
Least useful feature: industry
Feature: employment_status, Accuracy without: 0.6962457337883959, Difference: 0.0034129692832763903
Least useful feature: industry
Feature: lead_socre, Accuracy without: 0.6996587030716723, Difference: 0.0
Least useful feature: industry


In [144]:
#Questiuon 6 
C_values = [0.01, 0.1, 1, 10, 100]
best_accuracy = 0
best_C = None


for C in C_values:
    model = LogisticRegression(solver = 'liblinear',C=C,max_iter = 1000,random_state = 42)
    model.fit(X_train,y_train)
    y_val_pred = model.predict(X_val)
    acc = accuracy_score(y_val,y_val_pred)
    print(f"C ={C}, validation accuracy = {round(acc,3)}")
    if round(acc,3) > best_accuracy:
        best_accuracy = round(acc,3)
        best_C = C

print(f"Best C: {best_C}, Best accuracy: {best_accuracy}")

C =0.01, validation accuracy = 0.7
C =0.1, validation accuracy = 0.7
C =1, validation accuracy = 0.7
C =10, validation accuracy = 0.7
C =100, validation accuracy = 0.7
Best C: 0.01, Best accuracy: 0.7
